# SITCOMTN-096 M2 outer control loop - closed-loop mode


This is a notebook to analyze 



 - Technote:  https://sitcomtn-096.lsst.io/ <br> SITCOM-1105 
 https://jira.lsstcorp.org/browse/SITCOM-1105https://jira.lsstcorp.org/browse/SITCOM-1105 <br>

## 0. Import and Setup for the notebook


This is a part to import and set up for the notebook. <br>
It requires the following git repositories (and other requirements for those repos): <br>
ts_aos_utils:  https://github.com/lsst-ts/ts_aos_utils <br>
sitcom notebook_vandv: https://github.com/lsst-sitcom/notebooks_vandv <br>

In [ ]:
%matplotlib inline
import sys, time, os, asyncio, glob
from datetime import datetime
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
import pickle as pkl
from astropy.time import Time, TimeDelta
from pandas import DatetimeIndex
from scipy.interpolate import UnivariateSpline
from lsst_efd_client import EfdClient
from lsst.ts.aos.utils import DiagnosticsM2
from itertools import groupby
from matplotlib.lines import Line2D
from lsst.ts.aos.utils import EfdName

In [ ]:
import itertools as itt
import pandas as pd
import matplotlib.dates as mdates
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import math
import sys


from astropy import units as u
from astropy.time import Time, TimezoneInfo

from lsst.sitcom import vandv

If you use this notebook on 
1) summit: DiagnosticsM2(efd_name=EfdName.Summit)
2) idf : DiagnosticsM2(efd_name=EfdName.Idf)
3) usdf: DiagnosticsM2(efd_name=EfdName.Usdf)

In [ ]:
client = EfdClient("usdf_efd")

In [ ]:
total_axial_number = 72
total_tangent_number = 6

In [ ]:
m2 = DiagnosticsM2(efd_name=EfdName.Usdf)

## 1. Define functions
- actuator_force_range : find the time of range when each actuator starts and stops moving.
- force_statistics: get mean and standard deviations of applied, measured, demanded forces, and steps


In [ ]:
def actuator_force_range(
    applied_force_command: pd.core.series.Series, force: dict, i: int
):
    """Find the range of each actuactor

    Parameters
    -----------
    applied_force_command: `pandas.core.series.Series`
        Command for applying force on tangent/axial i actuator.
    force: `dict'
        tangent/axial force (all). Timestamp is private_sndStamp.
    i: int
        id of i actuator.

    Returns
    -------
    start_time_i: tai; datetime; numpy.float64
    end_time_i: tai; datetime; numpy.float64


    """

    applied_force_command_i = np.abs(applied_force_command) > 0
    ap_force_i = applied_force_command[applied_force_command_i]
    end_range = force["timestamp"][
        (np.abs(force["applied"][:, i]) <= 10)
        & (
            force["timestamp"]
            > force["timestamp"][np.abs(force["applied"][:, i]) >= 10][-1]
        )
    ][1]
    start_time_i = Time(ap_force_i.index[0]).utc.datetime.timestamp() - 5
    end_time_i = end_range + 10

    return start_time_i, end_time_i

In [ ]:
def force_statistics(
    applied_force_command: pd.core.series.Series,
    force: dict,
    demanded_force: pd.core.series.Series,
    step: np.typing.NDArray[np.float64],
    timestamp_step: np.typing.NDArray[np.float64],
    start_time_i: np.float64,
    end_time_i: np.float64,
    i: int,
):
    """Mean and standard deviations of applied forces, measured forces, demanded forces, and motor steps


    Parameters
    ----------
    applied_force_command: `pandas.core.series.Series`
        A command for applying force on tangent/axial i actuator.
    force: `dict'
        force on tangent/axial i actuator.
    demanded_force: `pandas.core.series.Series`
        force defined as LUT_force + applied_force + HP_correction.
    step: `numpy.ndarray'
        step on tangent/axial i actuator.
    time_stamp_step:
        time stamp for step
    start_time_i: numpy.float64
        time when the test *start* for tangent/axial i actuator
    end_time_i:numpy.float64
        time when the test *end* for tangent/axial i actuator
    i: int
        id of i actuator.


    Returns
    -------
    force_mean_std: `dict'

    """

    force_mean_std = {
        "ap_force": {"mean": [], "std": []},
        "m_force": {"mean": [], "std": []},
        "step": {"mean": [], "std": []},
        "dm_force": {"mean": [], "std": []},
    }

    applied_force_command_i = np.abs(applied_force_command) > 0
    ap_force_i = applied_force_command[applied_force_command_i]
    applied_forces = force["applied"][:, i]
    shifted_m_forces = force["measured"][:, i] - np.median(force["measured"][:, i])
    shifted_dm_forces = demanded_force - np.median(demanded_force)
    timestamp = force["timestamp"]

    for j in range(len(ap_force_i)):
        ap_start = Time(ap_force_i.index[j]).tai.datetime.timestamp()
        if j != len(ap_force_i.index) - 1:
            ap_end = Time(ap_force_i.index[j + 1]).tai.datetime.timestamp()
        else:
            ap_end = end_time_i

        # Criteria : applied forces > 10N  measured forces > 5N;
        ap_forces = np.where(
            (timestamp >= ap_start)
            & (timestamp <= ap_end)
            & ((np.abs(applied_forces) > 10))
        )
        m_forces = np.where(
            (timestamp >= ap_start)
            & (timestamp <= ap_end)
            & ((np.abs(shifted_m_forces) > 5))
        )

        dm_forces = np.where(
            (timestamp >= ap_start)
            & (timestamp <= ap_end)
            & ((np.abs(shifted_dm_forces) > 5))
        )

        if len(ap_forces[0]) > 0:
            force_mean_std["ap_force"]["mean"].append(
                np.mean(applied_forces[ap_forces[0]])
            )
            force_mean_std["ap_force"]["std"].append(
                np.std(applied_forces[ap_forces[0]])
            )

        # For some steps, some points were missing for applied forces/measured forces, so put NaN for their average.
        else:
            force_mean_std["ap_force"]["mean"].append(np.nan)
            force_mean_std["ap_force"]["std"].append(np.nan)

        if len(m_forces[0]) > 0:
            force_mean_std["m_force"]["mean"].append(
                np.mean(shifted_m_forces[m_forces[0]])
            )
            force_mean_std["m_force"]["std"].append(
                np.std(shifted_m_forces[m_forces[0]])
            )
            step_forces = np.where(
                (timestamp_step >= ap_start)
                & (timestamp_step < timestamp[m_forces[0][-1] + 1] - 1)
                & ((step > 60) | (step < -100))
            )

        else:
            force_mean_std["m_force"]["mean"].append(np.nan)
            force_mean_std["m_force"]["std"].append(np.nan)

        if len(dm_forces[0]) > 0:
            force_mean_std["dm_force"]["mean"].append(
                np.mean(shifted_dm_forces[dm_forces[0]])
            )
            force_mean_std["dm_force"]["std"].append(
                np.std(shifted_dm_forces[dm_forces[0]])
            )

        else:
            force_mean_std["dm_force"]["mean"].append(np.nan)
            force_mean_std["dm_force"]["std"].append(np.nan)

        if len(step_forces[0]) > 0:
            force_mean_std["step"]["mean"].append(np.mean(step[step_forces[0]]))
            force_mean_std["step"]["std"].append(np.std(step[step_forces[0]]))

        else:
            force_mean_std["step"]["mean"].append(np.nan)
            force_mean_std["step"]["std"].append(np.nan)

    return force_mean_std

## 2. M2 outer control loop - closed-loop mode

Applied Forces and Measured Forces on each Tangent link with respect to time

* Note
 -  Test Case: https://jira.lsstcorp.org/secure/Tests.jspa#/testCase/LVV-T1826 <br>
 -  Execution: https://jira.lsstcorp.org/secure/Tests.jspa#/testPlayer/testExecution/LVV-E2864 <br> 
 -  Chronograf: https://usdf-rsp.slac.stanford.edu/chronograf/sources/1/chronograf/data-explorer?query=SELECT%20%22steps1%22%2C%20%22steps3%22%2C%20%22steps5%22%2C%20%22steps0%22%2C%20%22steps2%22%2C%20%22steps4%22%20FROM%20%22efd%22.%22autogen%22.%22lsst.sal.MTM2.tangentActuatorSteps%22%20WHERE%20time%20%3E%20%3AdashboardTime%3A%20AND%20time%20%3C%20%3AupperDashboardTime%3A 

In [ ]:
test_case = "LVV-T1826"  # Test Case: https://jira.lsstcorp.org/secure/Tests.jspa#/testCase/LVV-T1826
test_exec = "LVV-E2864"  # Executed on 2023-06-14

t_duration = 1200
delta_t = 20

In [ ]:
t_start_tangent = "2023-06-14T21:48:27"
t_end_tangent = "2023-06-15T00:23:00"

In [ ]:
time_start_tangent = Time(t_start_tangent, scale="utc", format="isot")
time_end_tangent = Time(t_end_tangent, scale="utc", format="isot")

In [ ]:
data_step_tangent, time_step_tangent = await m2.get_data_step_tangent(
    time_start_tangent, time_end_tangent, realign_time=False
)

In [ ]:
force_axial, force_tangent = await m2.get_data_force(
    time_start_tangent, time_end_tangent
)

There is no function in DiagnosticsM2 to query the topic for "command_applyForces" yet. <br>
Before there is any change on it, query thru client. 

In [ ]:
# Read Applied forces command
applied_force_command_tangent = await client.select_time_series(
    "lsst.sal.MTM2.command_applyForces",
    "*",
    Time(time_start_tangent, format="isot", scale="utc"),
    Time(time_end_tangent, format="isot", scale="utc"),
)

### 1) Demanded forces, measured forces, and steps on each tangent link 

<b>The demanded force </b> for each actuator are derived as: <br>
demand_force = LUT_force (LUTGravity+LUTTemperature) + applied_force + hardpointCorrection <br>

This step is for:
1. Plot Demanded forces, measured forces, and steps on each tangent link with respect to time.


Note that: 
1. For demanded force and measured forces, they are shifted to the zero point using their medium values.
2. For A1, A3, and A5, they are actively controlled whereas A2, A4, and A6 are passively controlled.
   That is why no motor step change for A2, A4, and A6.
3. It seems that there are some missing points from telemetry data.

In [ ]:
for i in range(0, total_tangent_number):
    fig, ax = plt.subplots(1, 1, figsize=(10, 5))

    # 1-2. Step for i tangent actuator.
    step = data_step_tangent[:, i]
    # 1-3. applied force command on i tangent actuator.
    applied_force_command_i = np.abs(
        applied_force_command_tangent[str("".join(("tangent", str(i))))]
    )

    applied_force_command_arrow = applied_force_command_tangent[
        np.abs(applied_force_command_tangent[str("".join(("tangent", str(i))))]) > 0
    ]

    start_time_i, end_time_i = actuator_force_range(
        applied_force_command_i, force_tangent, i
    )

    # demand_force = LUT_force (LUTGravity+LUTTemperature) + applied_force + HP_correction
    demanded_force = (
        force_tangent["lutGravity"][:, i]
        + force_tangent["lutTemperature"][:, i]
        + force_tangent["applied"][:, i]
        + force_tangent["hardpointCorrection"][:, i]
    )

    # 1-1. Define measured forces (shifted with median values)

    start_time_a = np.where(force_tangent["timestamp"] >= start_time_i)[0][0]
    end_time_a = np.where(force_tangent["timestamp"] > end_time_i)[0][0]

    shifted_m_forces = force_tangent["measured"][
        start_time_a:end_time_a, i
    ] - np.median(force_tangent["measured"][start_time_a:end_time_a, i])


    ax.plot(
        force_tangent["timestamp"],
        demanded_force - np.median(demanded_force),
        ".-",
        label="Demanded Force",
        color="blue",
    )

    ax.plot(
        force_tangent["timestamp"][start_time_a:end_time_a],
        shifted_m_forces,
        ".--",
        color="red",
        label="Measured Force",
    )

    ax.scatter(
        applied_force_command_arrow["private_sndStamp"],
        [10] * len(applied_force_command_arrow["private_sndStamp"]),
        marker=r"$\downarrow$",
        s=500,
        color="m",
        linewidth=0.05,
    )

    ax.set_xlim([start_time_i, end_time_i])

    x_major_positions = [label.get_position()[0] for label in ax.get_xticklabels()]

    ax.set_xticks(ax.get_xticks().tolist())
    ax.set_xticklabels(
        [Time(ts, format="unix_tai").strftime("%H:%M") for ts in x_major_positions]
    )

    # ax.xaxis.set_major_formatter(mdates.DateFormatter("%Y-%m-%d %H:%M:%S UTC"))
    ax.grid(which="major", axis="both", linestyle="--")

    ax2 = ax.twinx()
    ax2.set_xlim([start_time_i, end_time_i])

    ax2.set_ylim([-1500, 1500])

    ax2.plot(time_step_tangent, step, ".-", label="Steps", color="#4daf4a")

    ax.set_xlabel("Time [UTC]", fontsize=12)

    line1 = Line2D([0], [0], label="Applied Force", linestyle="-", color="grey")
    line2 = Line2D([0], [0], label="Demanded Force", linestyle="-", color="blue")
    line3 = Line2D([0], [0], label="Measured Force", linestyle="--", color="red")
    line4 = Line2D([0], [0], label="Step", linestyle="-", color="g")
    line5 = Line2D([0], [0], label="Command to apply force", linestyle="-", color="m")

    handles = [line2, line3, line4, line5]
    ax.legend(handles=handles, loc="lower left", fontsize=12)
    # fig.tight_layout()

    x_lower_limit = Time(start_time_i, format="unix_tai").utc.to_datetime()
    x_upper_limit = Time(end_time_i, format="unix_tai").utc.to_datetime()

    title = (
        f"{test_case} - {test_exec}\n"
        "Demanded Forces, Measured Forces, and Steps on "
        f"A{i + 1} tangent link \n"
        # f" {tangent_steps.index[0].isoformat(timespec='seconds')[:-6]} -"
        # f" {tangent_steps.index[-1].isoformat(timespec='seconds')[:-6]}")
        f" {x_lower_limit.isoformat(timespec='seconds')} -"
        f" {x_upper_limit.isoformat(timespec='seconds')}"
    )
    ax.set_title(f"{title}")
    ax.set_ylabel("Forces (N)", fontsize=12)
    ax2.set_ylabel("Steps", fontsize=12)

    os.makedirs("plots", exist_ok=True)
    fig.savefig(f"./plots/Time_vs_Forces_and_Step_on_A{i+1}.png")

In [ ]:
# A specific case of tangential link A1. 

i=0


zoom_start, zoom_end = [] , []

#zoom in regions for each stamp 
zoom_start.append((Time("2023-06-14T21:51:40").utc.datetime.timestamp()))
zoom_end.append((Time("2023-06-14T21:53:45").utc.datetime.timestamp()))
zoom_start.append((Time("2023-06-14T21:53:30").utc.datetime.timestamp()))
zoom_end.append((Time("2023-06-14T21:55:00").utc.datetime.timestamp()))
zoom_start.append((Time("2023-06-14T21:55:00").utc.datetime.timestamp()))
zoom_end.append((Time("2023-06-14T21:56:40").utc.datetime.timestamp()))


fig, ax = plt.subplots(1, 3, figsize=(15, 5))

applied_force_command_i = np.abs(
    applied_force_command_tangent[str("".join(("tangent", str(i))))]
)

applied_force_command_arrow = applied_force_command_tangent[
    np.abs(applied_force_command_tangent[str("".join(("tangent", str(i))))]) > 0
]



start_time_i, end_time_i = actuator_force_range(
    applied_force_command_i, force_tangent, i
)

# demand_force = LUT_force (LUTGravity+LUTTemperature) + applied_force + HP_correction
demanded_force = (
    force_tangent["lutGravity"][:, i]
    + force_tangent["lutTemperature"][:, i]
    + force_tangent["applied"][:, i]
    + force_tangent["hardpointCorrection"][:, i]
)

# 1-1. Define measured forces (shifted with median values)

start_time_a = np.where(force_tangent["timestamp"] >= start_time_i)[0][0]
end_time_a = np.where(force_tangent["timestamp"] > end_time_i)[0][0]

shifted_m_forces = force_tangent["measured"][
    start_time_a:end_time_a, i
] - np.median(force_tangent["measured"][start_time_a:end_time_a, i])

#print(Time(t_start_tangent,format="unix_tai").strftime("%H:%M"))

ax[0].set_ylim([-10, 30])
ax[2].set_ylim([-20, 70])

line1 = Line2D([0], [0], label="Demanded Force", linestyle="-", color="blue")
line2 = Line2D([0], [0], label="Measured Force", linestyle="--", color="red")

handles = [line1, line2]

for j in range(0,3):
    ax[j].xaxis.set_major_locator(ticker.MaxNLocator(nbins=4))
    ax[j].xaxis.set_tick_params(direction='inout')
    ax[j].yaxis.set_tick_params(direction='inout')
    ax[j].set_xlim([zoom_start[j], zoom_end[j]])
    x_major_positions = [label.get_position()[0] for label in ax[j].get_xticklabels()]
    ax[j].set_xticks(ax[j].get_xticks().tolist())
    ax[j].set_xticklabels(
    [Time(ts, format="unix_tai").strftime("%H:%M:%S") for ts in x_major_positions])
    ax[j].grid(which="major", axis="both", linestyle="--")
    ax[j].set_xlabel("Time [UTC]", fontsize=12)
    ax[j].set_ylabel("Forces (N)", fontsize=12)

    ax[j].plot(
        force_tangent["timestamp"],
        demanded_force - np.median(demanded_force),
        ".-",
        label="Demanded Force",
        color="blue",
    )
    
    ax[j].plot(
        force_tangent["timestamp"][start_time_a:end_time_a],
        shifted_m_forces,
        ".--",
        color="red",
        label="Measured Force",
    )
    ax[j].legend(handles=handles, loc="lower left", fontsize=12)

    
# fig.tight_layout()


rect_x_l = Time("2023-06-14T21:52:05").utc.datetime.timestamp()
rect_x_r = Time("2023-06-14T21:53:20").utc.datetime.timestamp()

rect_x = [rect_x_l, rect_x_r, rect_x_r, rect_x_l, rect_x_l]
rect_y = [22, 22, 27, 27, 22]

ax[0].fill(rect_x, rect_y, label='Filled Rectangular Plot', alpha=0.5 , color="orange")

rect_x_l = Time("2023-06-14T21:55:25").utc.datetime.timestamp()
rect_x_r = Time("2023-06-14T21:56:15").utc.datetime.timestamp()

rect_x = [rect_x_l, rect_x_r, rect_x_r, rect_x_l, rect_x_l]
rect_y = [48, 48, 52, 52, 48]

ax[2].fill(rect_x, rect_y, label='Filled Rectangular Plot', alpha=0.5 , color="green")

ax[0].set_title("Interval for $\sigma$ Computation",fontsize=15)
ax[1].set_title("Inaccurate Telemetry for Sampling \n (No EFD Data for Applied Force)",fontsize=15)
ax[2].set_title("Interval for $\sigma$ Computation",fontsize=15)

fig.tight_layout()
os.makedirs("plots", exist_ok=True)
fig.savefig(f"./plots/Time_vs_Forces_and_Step_on_A{i+1}_Zoom_in.png")



### 2) Linear Regression for 1] Moter Steps vs. Demanded Forces 2] Moter steps vs. Measured Forces.

This cell is for
1. Get means and standard deviations for demanded forces, measured forces, and motor steps when force was applied on each tangential link ("plateaus" on the plots above) 
2. Plot step vs. demanded forces and Step vs. Measured forces with their Std Devs on the same figure
3. Fit linear regressions and get the RMS for each plot
4. Plot RMS w.r.t Step.  

The goals of this step are:
1. The force commands are accepted 
2. The EUI/EFD shows the commanded force is being applied after each command
3. Plot MTM2_tangentActuatorStepsN, MTM2.tangentForce.measuredN and plot MTM2_tangentActuatorSteps.stepsN - demanded forces derived using  and verify that the relation among the two variables is linear.

In [ ]:
x = np.arange(-1200, 1200, 0.5)

# As tangential Links 1, 3, 5 are only under active control and moving, check

for i in range(total_tangent_number):
    if i % 2 != 0:
            continue
        
    fig, axes = plt.subplots(2, 1, figsize=(6, 8), height_ratios=[4, 1])
    ax = axes[0]
    
    applied_force_command_i = np.abs(
        applied_force_command_tangent[str("".join(("tangent", str(i))))]
    )
    
    start_time_i, end_time_i = actuator_force_range(
        applied_force_command_i, force_tangent, i
    )
    
    # Zero point of Measured forces are shifted with their median values.
    shifted_m_forces = force_tangent["measured"][
        start_time_a:end_time_a, i
    ] - np.median(force_tangent["measured"][start_time_a:end_time_a, i])
    
    step = data_step_tangent[:, i]
    
    applied_forces = force_tangent["applied"][:, i]
    
    demanded_force = (
        force_tangent["lutGravity"][:, i]
        + force_tangent["lutTemperature"][:, i]
        + force_tangent["applied"][:, i]
        + force_tangent["hardpointCorrection"][:, i]
    )
    
    force_mean_std = force_statistics(
        applied_force_command_i,
        force_tangent,
        demanded_force,
        step,
        time_step_tangent,
        start_time_i,
        end_time_i,
        i,
    )
    
    # 1. Divide the section for applied forces and measured forces for each tangential link
    
    ap_idx = ~np.isnan(force_mean_std["step"]["mean"]) & ~np.isnan(
        force_mean_std["ap_force"]["mean"]
    )
    m_idx = ~np.isnan(force_mean_std["step"]["mean"]) & ~np.isnan(
        force_mean_std["m_force"]["mean"]
    )
    dm_idx = ~np.isnan(force_mean_std["step"]["mean"]) & ~np.isnan(
        force_mean_std["dm_force"]["mean"]
    )
    
    degree = 1
    
    # coefficient for linear fitting
    coefficients_ap = np.polyfit(
        np.array(force_mean_std["step"]["mean"])[ap_idx],
        np.array(force_mean_std["ap_force"]["mean"])[ap_idx],
        degree,
    )
    coefficients_m = np.polyfit(
        np.array(force_mean_std["step"]["mean"])[m_idx],
        np.array(force_mean_std["m_force"]["mean"])[m_idx],
        degree,
    )
    
    coefficients_dm = np.polyfit(
        np.array(force_mean_std["step"]["mean"])[dm_idx],
        np.array(force_mean_std["dm_force"]["mean"])[dm_idx],
        degree,
    )
    
    poly_ap = np.poly1d(coefficients_ap)
    poly_m = np.poly1d(coefficients_m)
    poly_dm = np.poly1d(coefficients_dm)
    
    y_pred_ap = poly_ap(np.array(force_mean_std["step"]["mean"])[ap_idx])
    y_pred_m = poly_m(np.array(force_mean_std["step"]["mean"])[m_idx])
    y_pred_dm = poly_dm(np.array(force_mean_std["step"]["mean"])[dm_idx])
    
    rms_ap = np.array(force_mean_std["ap_force"]["mean"])[ap_idx] - y_pred_ap
    rms_m = np.array(force_mean_std["m_force"]["mean"])[m_idx] - y_pred_m
    rms_dm = np.array(force_mean_std["dm_force"]["mean"])[dm_idx] - y_pred_dm
    
    m_ap, b_ap = np.polyfit(
        np.array(force_mean_std["step"]["mean"])[ap_idx],
        np.array(force_mean_std["ap_force"]["mean"])[ap_idx],
        1,
    )
    m_m, b_m = np.polyfit(
        np.array(force_mean_std["step"]["mean"])[m_idx],
        np.array(force_mean_std["m_force"]["mean"])[m_idx],
        1,
    )
    
    m_dm, b_dm = np.polyfit(
        np.array(force_mean_std["step"]["mean"])[dm_idx],
        np.array(force_mean_std["dm_force"]["mean"])[dm_idx],
        1,
    )
    
    
    #We only compare measured forces vs. step and demanded forces vs. step. 
    
    ax.plot(x, m_dm * x + b_dm, "-b")
    ax.plot(x, m_m * x + b_m, "--r")
    
    
    ax.errorbar(
        np.array(force_mean_std["step"]["mean"])[dm_idx],
        np.array(force_mean_std["dm_force"]["mean"])[dm_idx],
        fmt="bo",
        xerr=np.array(force_mean_std["step"]["std"])[dm_idx],
        yerr=np.array(force_mean_std["dm_force"]["std"])[dm_idx],
        label="Demanded Force",
    )
    
    
    ax.errorbar(
        np.array(force_mean_std["step"]["mean"])[m_idx],
        np.array(force_mean_std["m_force"]["mean"])[m_idx],
        fmt="ro",
        xerr=np.array(force_mean_std["step"]["std"])[m_idx],
        yerr=np.array(force_mean_std["m_force"]["std"])[m_idx],
        label="Measured Force",
    )
    
    
    
    ax.text(
        ax.get_xlim()[0] * 0.55 + ax.get_xlim()[1] * 0.45,
        ax.get_ylim()[0] * 0.88 + ax.get_ylim()[1] * 0.12,
        "RMS (measured force): " + format(np.sqrt(np.mean(rms_m**2)), ".3f"),
        bbox=dict(facecolor="white", alpha=0.5, edgecolor="none"),
        fontsize=12,
    )
    
    ax.text(
        ax.get_xlim()[0] * 0.55 + ax.get_xlim()[1] * 0.45,
        ax.get_ylim()[0] * 0.84 + ax.get_ylim()[1] * 0.16,
        "RMS (demanded force): " + format(np.sqrt(np.mean(rms_dm**2)), ".3f"),
        bbox=dict(facecolor="white", alpha=0.5, edgecolor="none"),
        fontsize=12,
    )
    
    ax.legend(fontsize=11, framealpha=1)  # ,bbox_to_anchor=(1.2, -0.2))
    ax.grid(which="major", axis="both", linestyle="--")
    
    x_lower_limit = Time(start_time_i, format="unix_tai").utc.to_datetime()
    x_upper_limit = Time(end_time_i, format="unix_tai").utc.to_datetime()
    
    title = (
        f"{test_case} - {test_exec}\n"
        "Measured & Demanded Forces vs. Steps on "
        "tangent link A"
        f"{i+1} \n"
        f" {x_lower_limit.isoformat(timespec='seconds')} -"
        f" {x_upper_limit.isoformat(timespec='seconds')}"
    )
    
    ax.set_title(f"{title}")
    
    ax.set_ylabel("Force (N)", fontsize=12)
    ax.set_xlabel("$\Delta$Steps (from Median)", fontsize=12)
    ax.grid(which="major", axis="both", linestyle="--")
    # ax.set_xlim([-1200, 1200])
    
    ax = axes[1]
    
    ax.errorbar(
        np.array(force_mean_std["step"]["mean"])[m_idx],
        rms_m,
        fmt="ro",
        xerr=np.array(force_mean_std["step"]["std"])[m_idx],
        yerr=np.array(force_mean_std["m_force"]["std"])[m_idx],
        label="Measured Force",
    )
    
    ax.errorbar(
        np.array(force_mean_std["step"]["mean"])[dm_idx],
        rms_dm,
        fmt="bo",
        xerr=np.array(force_mean_std["step"]["std"])[dm_idx],
        yerr=np.array(force_mean_std["dm_force"]["std"])[dm_idx],
        label="Demanded Force",
    )
    
    ax.axhline(y=0, linestyle=":", color="k")
    ax.grid(which="major", axis="both", linestyle="--")
    ax.set_ylabel("$\Delta$Force (N)", fontsize=12)
    ax.set_xlabel("$\Delta$Steps (from Median)", fontsize=12)
    ax.grid(which="major", axis="both", linestyle="--")
    # ax.set_xlim([-1200, 1200])
    
    ax.set_ylim(
        [
            min(np.concatenate((rms_m, rms_dm))) * 1.5,
            max((np.concatenate((rms_m, rms_dm))) * 1.5),
        ]
    )
    fig.tight_layout()
    os.makedirs("plots", exist_ok=True)
    fig.savefig(f"./plots/Time_vs_Forces_and_Step_on_A{i+1}.png")


## Let's try to exclude the outlier for A1. 

In [ ]:
x = np.arange(-1200, 1200, 0.5)

# As tangential Links 1, 3, 5 are only under active control and moving, check

i=0

fig, axes = plt.subplots(2, 1, figsize=(6, 8), height_ratios=[4, 1])
ax = axes[0]

applied_force_command_i = np.abs(
    applied_force_command_tangent[str("".join(("tangent", str(i))))]
)

start_time_i, end_time_i = actuator_force_range(
    applied_force_command_i, force_tangent, i
)

# Zero point of Measured forces are shifted with their median values.
shifted_m_forces = force_tangent["measured"][
    start_time_a:end_time_a, i
] - np.median(force_tangent["measured"][start_time_a:end_time_a, i])

step = data_step_tangent[:, i]

applied_forces = force_tangent["applied"][:, i]

demanded_force = (
    force_tangent["lutGravity"][:, i]
    + force_tangent["lutTemperature"][:, i]
    + force_tangent["applied"][:, i]
    + force_tangent["hardpointCorrection"][:, i]
)

force_mean_std = force_statistics(
    applied_force_command_i,
    force_tangent,
    demanded_force,
    step,
    time_step_tangent,
    start_time_i,
    end_time_i,
    i,
)

m_idx = ~np.isnan(force_mean_std["step"]["mean"]) & ~np.isnan(
    force_mean_std["m_force"]["mean"]) 
dm_idx = ~np.isnan(force_mean_std["step"]["mean"]) & ~np.isnan(
    force_mean_std["dm_force"]["mean"]) & (np.abs(np.array(force_mean_std["m_force"]["mean"])-np.array(force_mean_std["dm_force"]["mean"])) <20 )

degree = 1

print(m_idx, dm_idx)
coefficients_m = np.polyfit(
    np.array(force_mean_std["step"]["mean"])[m_idx],
    np.array(force_mean_std["m_force"]["mean"])[m_idx],
    degree,
)

coefficients_dm = np.polyfit(
    np.array(force_mean_std["step"]["mean"])[dm_idx],
    np.array(force_mean_std["dm_force"]["mean"])[dm_idx],
    degree,
)

poly_ap = np.poly1d(coefficients_ap)
poly_m = np.poly1d(coefficients_m)
poly_dm = np.poly1d(coefficients_dm)

y_pred_ap = poly_ap(np.array(force_mean_std["step"]["mean"])[ap_idx])
y_pred_m = poly_m(np.array(force_mean_std["step"]["mean"])[m_idx])
y_pred_dm = poly_dm(np.array(force_mean_std["step"]["mean"])[dm_idx])

rms_ap = np.array(force_mean_std["ap_force"]["mean"])[ap_idx] - y_pred_ap
rms_m = np.array(force_mean_std["m_force"]["mean"])[m_idx] - y_pred_m
rms_dm = np.array(force_mean_std["dm_force"]["mean"])[dm_idx] - y_pred_dm

m_ap, b_ap = np.polyfit(
    np.array(force_mean_std["step"]["mean"])[ap_idx],
    np.array(force_mean_std["ap_force"]["mean"])[ap_idx],
    1,
)
m_m, b_m = np.polyfit(
    np.array(force_mean_std["step"]["mean"])[m_idx],
    np.array(force_mean_std["m_force"]["mean"])[m_idx],
    1,
)

m_dm, b_dm = np.polyfit(
    np.array(force_mean_std["step"]["mean"])[dm_idx],
    np.array(force_mean_std["dm_force"]["mean"])[dm_idx],
    1,
)


#We only compare measured forces vs. step and demanded forces vs. step. 

ax.plot(x, m_dm * x + b_dm, "-b")
ax.plot(x, m_m * x + b_m, "--r")


ax.errorbar(
    np.array(force_mean_std["step"]["mean"])[dm_idx],
    np.array(force_mean_std["dm_force"]["mean"])[dm_idx],
    fmt="bo",
    xerr=np.array(force_mean_std["step"]["std"])[dm_idx],
    yerr=np.array(force_mean_std["dm_force"]["std"])[dm_idx],
    label="Demanded Force",
)


ax.errorbar(
    np.array(force_mean_std["step"]["mean"])[m_idx],
    np.array(force_mean_std["m_force"]["mean"])[m_idx],
    fmt="ro",
    xerr=np.array(force_mean_std["step"]["std"])[m_idx],
    yerr=np.array(force_mean_std["m_force"]["std"])[m_idx],
    label="Measured Force",
)

ax.errorbar(
    np.array(force_mean_std["step"]["mean"])[~dm_idx],
    np.array(force_mean_std["dm_force"]["mean"])[~dm_idx],
    fmt="bo",
    xerr=np.array(force_mean_std["step"]["std"])[~dm_idx],
    yerr=np.array(force_mean_std["dm_force"]["std"])[~dm_idx],
    zorder=1
)


ax.scatter(np.array(force_mean_std["step"]["mean"])[~dm_idx],np.array(force_mean_std["dm_force"]["mean"])[~dm_idx],
           marker='o', facecolors='white', edgecolors='blue', zorder=2,label="Outlier")


ax.text(
    ax.get_xlim()[0] * 0.55 + ax.get_xlim()[1] * 0.45,
    ax.get_ylim()[0] * 0.88 + ax.get_ylim()[1] * 0.12,
    "RMS (measured force): " + format(np.sqrt(np.mean(rms_m**2)), ".3f"),
    bbox=dict(facecolor="white", alpha=0.5, edgecolor="none"),
    fontsize=12,
)

ax.text(
    ax.get_xlim()[0] * 0.55 + ax.get_xlim()[1] * 0.45,
    ax.get_ylim()[0] * 0.84 + ax.get_ylim()[1] * 0.16,
    "RMS (demanded force): " + format(np.sqrt(np.mean(rms_dm**2)), ".3f"),
    bbox=dict(facecolor="white", alpha=0.5, edgecolor="none"),
    fontsize=12,
)

ax.legend(fontsize=11, framealpha=1)  # ,bbox_to_anchor=(1.2, -0.2))
ax.grid(which="major", axis="both", linestyle="--")

x_lower_limit = Time(start_time_i, format="unix_tai").utc.to_datetime()
x_upper_limit = Time(end_time_i, format="unix_tai").utc.to_datetime()

title = (
    f"{test_case} - {test_exec}\n"
    "Measured & Demanded Forces vs. Steps on "
    "tangent link A"
    f"{i+1} \n"
    f" {x_lower_limit.isoformat(timespec='seconds')} -"
    f" {x_upper_limit.isoformat(timespec='seconds')}"
)

ax.set_title(f"{title}")

ax.set_ylabel("Force (N)", fontsize=12)
ax.set_xlabel("$\Delta$Steps (from Median)", fontsize=12)
ax.grid(which="major", axis="both", linestyle="--")
# ax.set_xlim([-1200, 1200])

ax = axes[1]

ax.errorbar(
    np.array(force_mean_std["step"]["mean"])[m_idx],
    rms_m,
    fmt="ro",
    xerr=np.array(force_mean_std["step"]["std"])[m_idx],
    yerr=np.array(force_mean_std["m_force"]["std"])[m_idx],
    label="Measured Force",
)

ax.errorbar(
    np.array(force_mean_std["step"]["mean"])[dm_idx],
    rms_dm,
    fmt="bo",
    xerr=np.array(force_mean_std["step"]["std"])[dm_idx],
    yerr=np.array(force_mean_std["dm_force"]["std"])[dm_idx],
    label="Demanded Force",
)

ax.errorbar(
    np.array(force_mean_std["step"]["mean"])[~dm_idx],
    np.array(force_mean_std["dm_force"]["mean"])[~dm_idx]-m_dm*np.array(force_mean_std["step"]["mean"])[~dm_idx]+b_dm,
    fmt="bo",
    xerr=np.array(force_mean_std["step"]["std"])[~dm_idx],
    yerr=np.array(force_mean_std["dm_force"]["std"])[~dm_idx],
    zorder=1
)


ax.scatter(np.array(force_mean_std["step"]["mean"])[~dm_idx],np.array(force_mean_std["dm_force"]["mean"])[~dm_idx]-m_dm*np.array(force_mean_std["step"]["mean"])[~dm_idx]+b_dm,
           marker='o', facecolors='white', edgecolors='blue', zorder=2,label="Outlier")




ax.axhline(y=0, linestyle=":", color="k")
ax.grid(which="major", axis="both", linestyle="--")
ax.set_ylabel("$\Delta$Force (N)", fontsize=12)
ax.set_xlabel("$\Delta$Steps (from Median)", fontsize=12)
ax.grid(which="major", axis="both", linestyle="--")
# ax.set_xlim([-1200, 1200])

#ax.set_ylim([min(np.concatenate((rms_m, rms_dm))) * 1.5, max((np.concatenate((rms_m, rms_dm))) * 1.5)])
fig.tight_layout()
os.makedirs("plots", exist_ok=True)
fig.savefig(f"./plots/Time_vs_Forces_and_Step_on_A{i+1}_exclude.png")


## 3. Applied Forces and Measured Forces on each Axial link w.r.t time 


* Test Case: https://jira.lsstcorp.org/secure/Tests.jspa#/testCase/LVV-T2969https://jira.lsstcorp.org/secure/Tests.jspa#/testCase/LVV-T2969





In [ ]:
test_case = "LVV-T2969"
test_exec = "LVV-E3330"  # The test execution id is not clear.

In [ ]:
time_start_axial = "2023-11-08T20:11:40"
time_end_axial = "2023-11-08T20:50:07"

In [ ]:
t_start_axial = Time(time_start_axial, scale="utc", format="isot")
t_end_axial = Time(time_end_axial, scale="utc", format="isot")

In [ ]:
force_axial, force_tangent = await m2.get_data_force(t_start_axial, t_end_axial)
data_step_axial, time_step_axial = await m2.get_data_step_axial(
    t_start_axial, t_end_axial, realign_time=False
)

In [ ]:
axial_id = [9, 32, 70]
total_axial_number = len(axial_id)

In [ ]:
# Read Applied forces command
applied_force_command_axial = await client.select_time_series(
    "lsst.sal.MTM2.command_applyForces",
    "*",
    Time(time_start_axial, format="isot", scale="utc"),
    Time(time_end_axial, format="isot", scale="utc"),
)

### 1) Applied forces, measured forces, demanded forces, and steps on each axial link


In [ ]:
for i in axial_id:
    fig, ax = plt.subplots(1, 1, figsize=(10, 5))
    act_id = m2.get_alias_based_on_actuator_id(i)

    # 1-2. Step for i axial actuator.
    # step = data_step_axial[:, i]
    step = data_step_axial[:, i] - np.median(data_step_axial[:, i])

    # 1-3. applied force command on i axial actuator.
    applied_force_command_i = np.abs(
        applied_force_command_axial[str("".join(("axial", str(i))))]
    )

    applied_force_command_arrow = applied_force_command_axial[
        np.abs(applied_force_command_axial[str("".join(("axial", str(i))))]) > 0
    ]

    start_time_i, end_time_i = actuator_force_range(
        applied_force_command_i, force_axial, i
    )

    # 1-1. Define measured forces (shifted with median values)

    start_time_a = np.where(force_axial["timestamp"] >= start_time_i)[0][0]
    end_time_a = np.where(force_axial["timestamp"] > end_time_i)[0][0]

    shifted_m_forces = force_axial["measured"][start_time_a:end_time_a, i] - np.median(
        force_axial["measured"][start_time_a:end_time_a, i]
    )

    demanded_force = (
        force_axial["lutGravity"][:, i]
        + force_axial["lutTemperature"][:, i]
        + force_axial["applied"][:, i]
        + force_axial["hardpointCorrection"][:, i]
    )

    ax.plot(
        force_axial["timestamp"],
        demanded_force - np.median(demanded_force),
        ".-",
        label="Demanded Force",
        color="Blue",
    )

    ax.plot(
        force_axial["timestamp"][start_time_a:end_time_a],
        shifted_m_forces,
        ".--",
        color="red",
        label="Measured Force",
    )

    ax.scatter(
        applied_force_command_arrow["private_sndStamp"],
        [10] * len(applied_force_command_arrow["private_sndStamp"]),
        marker=r"$\downarrow$",
        s=500,
        color="m",
        linewidth=0.05,
    )

    ax.set_xlim([start_time_i, end_time_i])

    x_major_positions = [label.get_position()[0] for label in ax.get_xticklabels()]

    ax.set_xticks(ax.get_xticks().tolist())
    ax.set_xticklabels(
        [Time(ts, format="unix_tai").strftime("%H:%M") for ts in x_major_positions]
    )

    # ax.xaxis.set_major_formatter(mdates.DateFormatter("%Y-%m-%d %H:%M:%S UTC"))
    ax.grid(which="major", axis="both", linestyle="--")

    ax2 = ax.twinx()
    ax2.set_xlim([start_time_i, end_time_i])

    # ax2.set_ylim([-1500, 1500])

    ax2.plot(time_step_axial, step, ".-", label="Steps", color="g")

    ax.set_xlabel("Time [UTC]", fontsize=12)

    line1 = Line2D([0], [0], label="Applied Force", linestyle="-", color="grey")
    line2 = Line2D([0], [0], label="Demanded Force", linestyle="-", color="blue")
    line3 = Line2D([0], [0], label="Measured Force", linestyle="-", color="red")
    line4 = Line2D([0], [0], label="Step", linestyle="-", color="g")
    line5 = Line2D([0], [0], label="Command to apply force", linestyle="-", color="m")

    handles = [line2, line3, line4, line5]
    ax.legend(handles=handles, loc="lower left", fontsize=12)
    # fig.tight_layout()

    x_lower_limit = Time(start_time_i, format="unix_tai").utc.to_datetime()
    x_upper_limit = Time(end_time_i, format="unix_tai").utc.to_datetime()

    title = (
        f"{test_case} - {test_exec}\n"
        "Applied Forces, Measured Forces, and Steps on "
        f"{act_id} axial link \n"
        # f" {tangent_steps.index[0].isoformat(timespec='seconds')[:-6]} -"
        # f" {tangent_steps.index[-1].isoformat(timespec='seconds')[:-6]}")
        f" {x_lower_limit.isoformat(timespec='seconds')} -"
        f" {x_upper_limit.isoformat(timespec='seconds')}"
    )
    ax.set_title(f"{title}")
    ax.set_ylabel("Forces (N)", fontsize=12)
    ax2.set_ylabel("$\Delta$Steps (from Median)", fontsize=12)
    os.makedirs("plots", exist_ok=True)
    fig.savefig(f"./plots/Time_vs_Forces_and_Step_on_" f"{act_id}.png")

### 2) Linear Regression for 1] Moter Steps vs. Demanded Forces 2] Moter steps vs. Measured Forces.

This cell is for the same process as step 2 above but for the axial actuators B10, C3, and D17. 


In [ ]:
x = np.arange(-3000, 3000, 0.5)

for i in axial_id:
    fig, axes = plt.subplots(2, 1, figsize=(6, 8), height_ratios=[4, 1])
    ax = axes[0]
    act_id = m2.get_alias_based_on_actuator_id(i)

    # 1. Step for i axial actuator.
    # step = data_step_axial[:, i]
    step = data_step_axial[:, i] - np.median(data_step_axial[:, i])
    # 1-3. applied force command on i axial actuator.
    applied_force_command_i = np.abs(
        applied_force_command_axial[str("".join(("axial", str(i))))]
    )

    start_time_i, end_time_i = actuator_force_range(
        applied_force_command_i, force_axial, i
    )

    demanded_force = (
        force_axial["lutGravity"][:, i]
        + force_axial["lutTemperature"][:, i]
        + force_axial["applied"][:, i]
        + force_axial["hardpointCorrection"][:, i]
    )
    
    force_mean_std = force_statistics(
        applied_force_command_i,
        force_axial,
        demanded_force,
        step,
        time_step_axial,
        start_time_i,
        end_time_i,
        i,
    )

    ap_idx = ~np.isnan(force_mean_std["step"]["mean"]) & ~np.isnan(
        force_mean_std["ap_force"]["mean"]
    )
    m_idx = ~np.isnan(force_mean_std["step"]["mean"]) & ~np.isnan(
        force_mean_std["m_force"]["mean"]
    )

    dm_idx = ~np.isnan(force_mean_std["step"]["mean"]) & ~np.isnan(
        force_mean_std["dm_force"]["mean"]
    )

    degree = 1
    coefficients_ap = np.polyfit(
        np.array(force_mean_std["step"]["mean"])[ap_idx],
        np.array(force_mean_std["ap_force"]["mean"])[ap_idx],
        degree,
    )
    coefficients_m = np.polyfit(
        np.array(force_mean_std["step"]["mean"])[m_idx],
        np.array(force_mean_std["m_force"]["mean"])[m_idx],
        degree,
    )

    # coefficients between demanding force and steps
    coefficients_dm = np.polyfit(
        np.array(force_mean_std["step"]["mean"])[m_idx],
        np.array(force_mean_std["dm_force"]["mean"])[dm_idx],
        degree,
    )

    poly_ap = np.poly1d(coefficients_ap)
    poly_m = np.poly1d(coefficients_m)
    poly_dm = np.poly1d(coefficients_dm)

    y_pred_ap = poly_ap(np.array(force_mean_std["step"]["mean"])[ap_idx])
    y_pred_m = poly_m(np.array(force_mean_std["step"]["mean"])[m_idx])
    y_pred_dm = poly_m(np.array(force_mean_std["step"]["mean"])[dm_idx])

    rms_ap = np.array(force_mean_std["ap_force"]["mean"])[ap_idx] - y_pred_ap
    rms_m = np.array(force_mean_std["m_force"]["mean"])[m_idx] - y_pred_m
    rms_dm = np.array(force_mean_std["dm_force"]["mean"])[dm_idx] - y_pred_dm

    m_ap, b_ap = np.polyfit(
        np.array(force_mean_std["step"]["mean"])[ap_idx],
        np.array(force_mean_std["ap_force"]["mean"])[ap_idx],
        1,
    )
    m_m, b_m = np.polyfit(
        np.array(force_mean_std["step"]["mean"])[m_idx],
        np.array(force_mean_std["m_force"]["mean"])[m_idx],
        1,
    )

    m_dm, b_dm = np.polyfit(
        np.array(force_mean_std["step"]["mean"])[m_idx],
        np.array(force_mean_std["dm_force"]["mean"])[dm_idx],
        1,
    )

    ax.plot(x, m_dm * x + b_dm, "-b")
    ax.plot(x, m_m * x + b_m, "--r")

  

    ax.errorbar(
        np.array(force_mean_std["step"]["mean"])[m_idx],
        np.array(force_mean_std["m_force"]["mean"])[m_idx],
        fmt="bo",
        xerr=np.array(force_mean_std["step"]["std"])[dm_idx],
        yerr=np.array(force_mean_std["dm_force"]["std"])[dm_idx],
        label="Demanded Force",
    )
    
    ax.errorbar(
        np.array(force_mean_std["step"]["mean"])[m_idx],
        np.array(force_mean_std["m_force"]["mean"])[m_idx],
        fmt="ro",
        xerr=np.array(force_mean_std["step"]["std"])[m_idx],
        yerr=np.array(force_mean_std["m_force"]["std"])[m_idx],
        label="Measured Force",
    )



    ax.text(
        ax.get_xlim()[0] * 0.95 + ax.get_xlim()[1] * 0.05,
        ax.get_ylim()[0] * 0.88 + ax.get_ylim()[1] * 0.12,
        "RMS (measured force): " + format(np.sqrt(np.mean(rms_m**2)), ".3f"),
        bbox=dict(facecolor="white", alpha=0.5, edgecolor="none"),
        fontsize=12,
    )

    ax.text(
        ax.get_xlim()[0] * 0.95 + ax.get_xlim()[1] * 0.05,
        ax.get_ylim()[0] * 0.84 + ax.get_ylim()[1] * 0.16,
        "RMS (demanded force): " + format(np.sqrt(np.mean(rms_dm**2)), ".3f"),
        bbox=dict(facecolor="white", alpha=0.5, edgecolor="none"),
        fontsize=12,
    )

    ax.legend(fontsize=11, framealpha=1)  # ,bbox_to_anchor=(1.2, -0.2))
    ax.grid(which="major", axis="both", linestyle="--")

    x_lower_limit = Time(start_time_i, format="unix_tai").utc.to_datetime()
    x_upper_limit = Time(end_time_i, format="unix_tai").utc.to_datetime()

    title = (
        f"{test_case} - {test_exec}\n"
        "Applied Forces, Measured Forces, and Steps on "
        f"{act_id} axial link \n"
        f" {x_lower_limit.isoformat(timespec='seconds')} -"
        f" {x_upper_limit.isoformat(timespec='seconds')}"
    )

    ax.set_title(f"{title}")

    ax.set_ylabel("Force (N)", fontsize=12)
    ax.set_xlabel("$\Delta$Steps (from Median)", fontsize=12)
    ax.grid(which="major", axis="both", linestyle="--")
    # ax.set_xlim([-1200, 1200])

    ax = axes[1]

    ax.errorbar(
        np.array(force_mean_std["step"]["mean"])[m_idx],
        rms_m,
        fmt="ro",
        xerr=np.array(force_mean_std["step"]["std"])[m_idx],
        yerr=np.array(force_mean_std["m_force"]["std"])[m_idx],
        label="Measured Force",
    )

    ax.errorbar(
        np.array(force_mean_std["step"]["mean"])[dm_idx],
        rms_dm,
        fmt="bo",
        xerr=np.array(force_mean_std["step"]["std"])[dm_idx],
        yerr=np.array(force_mean_std["dm_force"]["std"])[dm_idx],
        label="Demanded Force",
    )

    ax.axhline(y=0, linestyle=":", color="k")
    ax.grid(which="major", axis="both", linestyle="--")
    ax.set_ylabel("$\Delta$Force (N)", fontsize=12)
    ax.set_xlabel("$\Delta$Steps (from Median)", fontsize=12)
    ax.grid(which="major", axis="both", linestyle="--")
    # ax.set_xlim([-1200, 1200])

    ax.set_ylim(
        [
            min(np.concatenate((rms_m, rms_dm))) * 1.5,
            max((np.concatenate((rms_m, rms_dm))) * 1.5),
        ]
    )
    fig.tight_layout()
    os.makedirs("plots", exist_ok=True)
    fig.savefig(f"./plots/Step_vs_Forces_on_" f"{act_id}.png")